In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# cur_dir = "/kaggle/working"
csv_file_path = "/kaggle/input/online-shoppers-intention/online_shoppers_intention.csv"

In [4]:
df=pd.read_csv(csv_file_path)

In [5]:
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [6]:
df['VisitorType'].value_counts()

Returning_Visitor    10551
New_Visitor           1694
Other                   85
Name: VisitorType, dtype: int64

In [7]:
df['Month'].value_counts()

May     3364
Nov     2998
Mar     1907
Dec     1727
Oct      549
Sep      448
Aug      433
Jul      432
June     288
Feb      184
Name: Month, dtype: int64

In [8]:
df['VisitorType'].value_counts()

Returning_Visitor    10551
New_Visitor           1694
Other                   85
Name: VisitorType, dtype: int64

In [9]:
df.isnull().sum()

Administrative             0
Administrative_Duration    0
Informational              0
Informational_Duration     0
ProductRelated             0
ProductRelated_Duration    0
BounceRates                0
ExitRates                  0
PageValues                 0
SpecialDay                 0
Month                      0
OperatingSystems           0
Browser                    0
Region                     0
TrafficType                0
VisitorType                0
Weekend                    0
Revenue                    0
dtype: int64

In [10]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [11]:
df['Month'] = le.fit_transform(df.Month.values)
df['VisitorType'] = le.fit_transform(df.VisitorType.values)
df['Weekend'] = le.fit_transform(df.Weekend.values)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12330 non-null  int64  
 1   Administrative_Duration  12330 non-null  float64
 2   Informational            12330 non-null  int64  
 3   Informational_Duration   12330 non-null  float64
 4   ProductRelated           12330 non-null  int64  
 5   ProductRelated_Duration  12330 non-null  float64
 6   BounceRates              12330 non-null  float64
 7   ExitRates                12330 non-null  float64
 8   PageValues               12330 non-null  float64
 9   SpecialDay               12330 non-null  float64
 10  Month                    12330 non-null  int64  
 11  OperatingSystems         12330 non-null  int64  
 12  Browser                  12330 non-null  int64  
 13  Region                   12330 non-null  int64  
 14  TrafficType           

In [13]:
# Normalizing the columns

In [14]:
from sklearn.preprocessing import MinMaxScaler

float_columns= df.select_dtypes(include=['float64']).columns


In [15]:
float_columns=list(float_columns)

In [16]:
float_columns.remove('SpecialDay')

In [17]:
float_columns.append('ProductRelated')

In [18]:
float_columns

['Administrative_Duration',
 'Informational_Duration',
 'ProductRelated_Duration',
 'BounceRates',
 'ExitRates',
 'PageValues',
 'ProductRelated']

In [19]:
scaler=MinMaxScaler()
df[float_columns]=scaler.fit_transform(df[float_columns])

In [20]:
df

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.000000,0,0.0,0.001418,0.000000,1.000000,1.000000,0.000000,0.0,2,1,1,1,1,2,0,False
1,0,0.000000,0,0.0,0.002837,0.001000,0.000000,0.500000,0.000000,0.0,2,2,2,1,2,2,0,False
2,0,0.000000,0,0.0,0.001418,0.000000,1.000000,1.000000,0.000000,0.0,2,4,1,9,3,2,0,False
3,0,0.000000,0,0.0,0.002837,0.000042,0.250000,0.700000,0.000000,0.0,2,3,2,2,4,2,0,False
4,0,0.000000,0,0.0,0.014184,0.009809,0.100000,0.250000,0.000000,0.0,2,3,3,1,4,2,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,3,0.042663,0,0.0,0.075177,0.027883,0.035714,0.145153,0.033839,0.0,1,4,6,1,1,2,1,False
12326,0,0.000000,0,0.0,0.007092,0.007280,0.000000,0.106667,0.000000,0.0,7,3,2,1,8,2,1,False
12327,0,0.000000,0,0.0,0.008511,0.002880,0.416667,0.433333,0.000000,0.0,7,3,2,1,13,2,1,False
12328,4,0.022067,0,0.0,0.021277,0.005408,0.000000,0.105263,0.000000,0.0,7,2,2,3,11,2,0,False


In [21]:

# Split the data into features and target variable
X = df.drop(['Revenue', 'Weekend', 'Informational_Duration'], axis=1)
y_revenue = df['Revenue']
y_weekend = df['Weekend']
informational_duration = df['Informational_Duration']

# Split the data into training and testing sets
X_train, X_test, y_revenue_train, y_revenue_test, y_weekend_train, y_weekend_test, informational_duration_train, informational_duration_test = train_test_split(X, y_revenue, y_weekend, informational_duration, test_size=0.2, random_state=42)



In [22]:
# Initialize and train the Random Forest model for Revenue prediction
revenue_model = RandomForestClassifier()
revenue_model.fit(X_train, y_revenue_train)

# Predict revenue on the testing set
revenue_predictions = revenue_model.predict(X_test)

# Calculate accuracy for revenue predictions
revenue_accuracy = accuracy_score(y_revenue_test, revenue_predictions)



In [23]:
revenue_accuracy

0.8917274939172749

In [24]:
# Initialize and train the Random Forest model for Weekend prediction
weekend_model = RandomForestClassifier()
weekend_model.fit(X_train, y_weekend_train)

# Predict weekend on the testing set
weekend_predictions = weekend_model.predict(X_test)

# Calculate accuracy for weekend predictions
weekend_accuracy = accuracy_score(y_weekend_test, weekend_predictions)

# Print accuracy scores

print("Weekend Accuracy:", weekend_accuracy)




Weekend Accuracy: 0.7627737226277372


In [25]:
df

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.000000,0,0.0,0.001418,0.000000,1.000000,1.000000,0.000000,0.0,2,1,1,1,1,2,0,False
1,0,0.000000,0,0.0,0.002837,0.001000,0.000000,0.500000,0.000000,0.0,2,2,2,1,2,2,0,False
2,0,0.000000,0,0.0,0.001418,0.000000,1.000000,1.000000,0.000000,0.0,2,4,1,9,3,2,0,False
3,0,0.000000,0,0.0,0.002837,0.000042,0.250000,0.700000,0.000000,0.0,2,3,2,2,4,2,0,False
4,0,0.000000,0,0.0,0.014184,0.009809,0.100000,0.250000,0.000000,0.0,2,3,3,1,4,2,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12325,3,0.042663,0,0.0,0.075177,0.027883,0.035714,0.145153,0.033839,0.0,1,4,6,1,1,2,1,False
12326,0,0.000000,0,0.0,0.007092,0.007280,0.000000,0.106667,0.000000,0.0,7,3,2,1,8,2,1,False
12327,0,0.000000,0,0.0,0.008511,0.002880,0.416667,0.433333,0.000000,0.0,7,3,2,1,13,2,1,False
12328,4,0.022067,0,0.0,0.021277,0.005408,0.000000,0.105263,0.000000,0.0,7,2,2,3,11,2,0,False


In [26]:
informational_duration_test

8916     0.000000
772      0.092395
12250    0.000000
7793     0.000000
6601     0.287835
           ...   
295      0.000000
3694     0.000000
4974     0.009022
10383    0.003726
2611     0.000000
Name: Informational_Duration, Length: 2466, dtype: float64

In [27]:

# Use the trained model to predict Informational_Duration
from sklearn.ensemble import RandomForestRegressor
informational_duration_model = RandomForestRegressor()
informational_duration_model.fit(X_train, informational_duration_train)



RandomForestRegressor()

In [28]:

# Predict weekend on the testing set
informational_duration_prediction = informational_duration_model.predict(X_test)


In [29]:

# Calculate MAE for weekend predictions
from sklearn.metrics import mean_absolute_error

mean_absolute_error(informational_duration_test, informational_duration_prediction)

0.012107677393186383